In [15]:
import nltk
import pandas as pd
from nltk.corpus import stopwords
import string
from nltk.tokenize import word_tokenize
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE

In [16]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zhuko\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zhuko\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
stop_words = set(stopwords.words('english'))

In [18]:
import pandas as pd
df = pd.read_json('places.json')

In [19]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [21]:
df = df[~df['property_description'].str.strip().eq('')]

In [22]:
df.head()

,id,name,url,property_date_up_to_date,property_tags_text,property_hours_status,property_omment_for_everyone,property_data_manager,property_phone_whatsapp_status,property_tag_old_2,property_email,property_hours_of_operation,property_kp,property_photo_google_drive,property_,property_location,property_link_to_their_site,property_comment_for_additional_information,property_tags_status,property_statistics,property_vip,property_data_manager_cr,property_features_text,property_description,property_photo_status,property_main_tags_old,property_additional_information_old,property_created_time,property_description_comment,property_sales,property_web_status,property_link_to_vip_page,property_web,property_type,property_comments_marina,property_created_date,property_google_map_status,property_fire_base_status,property_google_map,property_whats_app_number,property_description_status,property_main_tags,property_tags_old,property_location_status,property_plan,property_tags,property_menu,property_phone_number,property_owner_manager,property_features,property_links_to_social_media,property_main_tags_text,property_link_to_guider,property_name
5,9bf91cea-4a04-4e84-8b4f-4d0f26132745,SUR,https://www.notion.so/SUR-9bf91cea4a044e848b4f4d0f26132745,None,,Done,,Иван Черников,Done,,cdpcoco@gruponosara.com,"Monday - Friday 7:30 AM - 5:00 PM, \nSaturday 7:30 AM - 1:00 PM, \nSunday - Сlosed",False,https://drive.google.com/drive/folders/1tpe4WUskSK9G5T0NuVA0wmaAh0JaQY2A,None,Playa del Coco,https://www.gruposur.com/,"Магазин в котором основной товар это краска, так же есть товары для ремонта, у них два сайта (тот что указан в поле сайт это сайт компании) есть еще сайт магазина (https://www.surenlinea.cr/)",Done,[],False,[Марина Богода],NaN,"Store SUR for a wide range of repair and construction products, including paint and painting accessories.",Исходники загружены на Диск,"[Store, Hardware Store]",,2024-09-18 11:45:00+00:00,,[],Web готов,None,sur_coco,Stores,,2024-09-18T11:45:00.000Z,Done,Не готово,https://maps.app.goo.gl/Yy8V2VDCaMYwVc6d9,+506 8981-9898,Done,Stores Hardware Store,[],Done,Free,"[None, None, None, None]",,+506 4082-5573,,Store,https://www.facebook.com/p/Centro-de-Pinturas-Sur-100063443509462/,NaN,None,SUR
7,34565cec-1181-4e4d-8eca-ba5614ec65d2,Pulmitan de Liberia,https://www.notion.so/Pulmitan-de-Liberia-34565cec11814e4d8ecaba5614ec65d2,None,,No,"Автобусная станция транспортной компании, есть автобусная станция и есть служба доставки, их нужно занести как разные объекты, обязательно нужно написать что для уточнения расписания нужно писать в WhatsApp",Иван Черников,Done,,,,False,https://drive.google.com/drive/folders/1Tml2Xg-PmetafwmKKfgwgzhi0b2A6iPS,None,Liberia,None,,Нужен тег,[],False,[Марина Богода],NaN,"Pulmitan de Liberia is a spacious bus station for comfortable waiting for buses. It has all the necessary facilities, including the possibility to buy a snack and a drink. For more information on the schedule, write to WhatsApp.",Исходники загружены на Диск,[],,2024-09-18 08:55:00+00:00,,[],Web готов,None,pulmitan_de_liberia,Adventures,,2024-09-18T08:55:00.000Z,Done,Не готово,https://maps.app.goo.gl/gREanUdGfc7JLYAr7,+506 6034 2578,Нужно описание,None,[],Done,Free,[],,+506 2222 0610,,None,,NaN,None,Pulmitan de Liberia
8,1559af3f-0b5f-4b3f-b315-24e60b860198,Super Baterias,https://www.notion.so/Super-Baterias-1559af3f0b5f4b3fb31524e60b860198,None,,Done,,Иван Черников,Done,,,Monday - Saturday 7:00 AM - 7:00 PM \nSunday 8:00 AM - 4:00 PM,False,https://drive.google.com/drive/folders/1yhujGzP677nj9Av8bY7cQ-brST7x3uxz,⭐️⭐️⭐️,Liberia,https://superb.superbaterias.com/,,In progress,[],False,[Марина Богода],NaN,"Super Baterias is a store specializing in the sale of car batteries and other energy storage solutions. They also offer battery recycling services, buying used batteries at a low price.\n\n• Battery diagnosis, installation, and recharge services\n• Express service and roadside assistance\n• Battery recycling available",Исходники загружены на Д

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 176 entries, 5 to 222
Data columns (total 54 columns):
 #   Column                                       Non-Null Count  Dtype              
---  ------                                       --------------  -----              
 0   id                                           176 non-null    object             
 1   name                                         176 non-null    object             
 2   url                                          176 non-null    object             
 3   property_date_up_to_date                     42 non-null     object             
 4   property_tags_text                           176 non-null    object             
 5   property_hours_status                        176 non-null    object             
 6   property_omment_for_everyone                 176 non-null    object             
 7   property_data_manager                        60 non-null     object             
 8   property_phone_whatsapp_status     

In [104]:
# Функция для предобработки текста
def preprocess_text(text):
    # Преобразование текста в нижний регистр
    text = text.lower()
    
    # Создание набора символов для удаления, включая '•'
    punctuation_to_remove = string.punctuation + '•'
    
    # Удаление знаков препинания
    text = text.translate(str.maketrans('', '', punctuation_to_remove))
    
    # Токенизация и удаление стоп-слов
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    
    # Обратное преобразование в строку
    return ' '.join(tokens)

# Применение функции к столбцу 'reutersNLTK'
df['property_description'] = df['property_description'].apply(preprocess_text)

In [105]:
guider_corpus = list(df['property_description'].apply(word_tokenize))

In [118]:
n = 3
train_data, padded_sents = padded_everygram_pipeline(n, guider_corpus)

In [119]:
model = MLE(n) 
model.fit(train_data, padded_sents)

In [120]:
context = ("bar","restaurant")

In [121]:
print("Следующее слово:", model.generate(5, context, random_seed=42))

Следующее слово: ['offers', 'dining', 'experience', 'delightful', 'cuisine']


In [122]:
context_counts = model.context_counts(model.vocab.lookup(context))
sorted_words = sorted(context_counts.items(), key=lambda item: item[1], reverse=True)
top_5_words = [word for word, count in sorted_words[:5]]

In [123]:
print("Топ-5 следующих слов для заданного контекста:", top_5_words)

Топ-5 следующих слов для заданного контекста: ['people', 'offers', 'everything']


In [124]:
# Проверка наличия контекста в корпусе
context_occurrences = sum(1 for sent in guider_corpus if context in zip(sent, sent[1:]))
print("Частота контекста в корпусе:", context_occurrences)

Частота контекста в корпусе: 3


In [125]:
context = ("coffee","shop")

In [126]:
print("Следующее слово:", model.generate(5, context, random_seed=42))

Следующее слово: ['founded', 'venezuelans', 'addition', 'food', 'blend']


In [127]:
context_counts = model.context_counts(model.vocab.lookup(context))
sorted_words = sorted(context_counts.items(), key=lambda item: item[1], reverse=True)
top_5_words = [word for word, count in sorted_words[:5]]

In [128]:
print("Топ-5 следующих слов для заданного контекста:", top_5_words)

Топ-5 следующих слов для заданного контекста: ['bistro', 'find', 'founded', 'seating']


In [129]:
# Проверка наличия контекста в корпусе
context_occurrences = sum(1 for sent in guider_corpus if context in zip(sent, sent[1:]))
print("Частота контекста в корпусе:", context_occurrences)

Частота контекста в корпусе: 4


In [106]:
n = 2
train_data, padded_sents = padded_everygram_pipeline(n, guider_corpus)

In [107]:
model = MLE(n) 
model.fit(train_data, padded_sents)

In [114]:
context='market'

In [115]:
print("Следующее слово:", model.generate(5, context, random_seed=42))

Следующее слово: ['one', 'costa', 'rica', 'delivers', 'premiumquality']


In [116]:
# Функция для получения топ-5 слов для заданного контекста
def get_top_5_next_words(model, context):
    # Получаем словарь вероятностей для всех слов, следующих за контекстом
    context = [context]
    candidates = model.context_counts(model.vocab.lookup(context))
    # Сортируем по вероятности и берем топ-5
    top_5 = sorted(candidates.items(), key=lambda item: item[1], reverse=True)[:5]
    return [word for word, count in top_5]

In [117]:
# Пример использования
 # пример слова-контекста
top_5_words = get_top_5_next_words(model, context)
print(f"Топ-5 слов после '{context}': {top_5_words}")

Топ-5 слов после 'market': ['convenience', 'great', 'local', 'provides', 'every']
